In [1]:
import os
import gc
import io
import sys
import keras
import random
from glob import glob
from constants import *
from keras import backend as K
import matplotlib.pyplot as plt
from utils import buildMagicPoint
from google.protobuf import text_format
from tensorboard.plugins import projector
from tensorflow.python.platform import gfile
from data_preparation import DataPreparation
from metrics import CornerDetectionAveragePrecision

2025-03-14 23:14:01.964816: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1741976041.978965    1591 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1741976041.984574    1591 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-03-14 23:14:02.001108: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
I0000 00:00:1741976044.913425    1591 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 55

In [2]:
# Defining paths
project_dir = "/mnt/c/Users/SohaibWaheed/Desktop/Personal_Projects/Final_SuperPoint"
dataset_dir = project_dir + "/dataset/tfrecords"
model_dir = project_dir + "/nd_he_normal_standardized_saved_models" #model_dir = project_dir + "/saved_models"
log_dir = project_dir + "/nd_he_normal_standardized_logs" #log_dir = project_dir + "/logs"
validation_tfrecord = dataset_dir + "/valid/valid_record_no_1.tfrecord"
train_log_dir = log_dir + '/train'
valid_log_dir = log_dir + '/valid'
projector_config_path = train_log_dir + "/projector_config.pbtxt"
embeddings_path = train_log_dir + "/embeddings.ckpt"

In [5]:
train_summary_writer = tf.summary.create_file_writer(train_log_dir)
valid_summary_writer = tf.summary.create_file_writer(valid_log_dir)

In [6]:
# To calculate the round number to start from
model_files = os.listdir(model_dir)
projector_config = projector.ProjectorConfig()

if len(model_files) == 0:
    roundStart = 1
    # Initialzing model if there is no previous model found    
    magicPoint = buildMagicPoint(input_shape=MP_INPUT_SHAPE)
    
    magicPoint.compile(
        optimizer=keras.optimizers.Adam(),
        loss=keras.losses.SparseCategoricalCrossentropy()
    )
else:
    roundStart = max([int(file[:-6].split("_")[-1]) for file in model_files]) + 1
    last_model_file = f"magicPoint_{roundStart-1}.keras"
    # Loading saved model
    magicPoint = keras.models.load_model(model_dir + "/" + last_model_file)
    # Delete previous files
    model_files.pop(model_files.index(last_model_file))
    '''for file in model_files:
        os.remove(model_dir + "/" + file)'''
        
    # To Load the existing .pbtxt file
    with gfile.GFile(projector_config_path, 'r') as f:
        text_format.Merge(f.read(), projector_config)
        

roundEnd = 445
# Initialize the dataset
dataPreparation = DataPreparation(MP_BATCH_SIZE)

# Validation Dataset
validDataset = dataPreparation.loadDataset([validation_tfrecord])
validDatasetIterator = validDataset.__iter__()
    

metric = CornerDetectionAveragePrecision()


/home/sohaibwaheed/anaconda3/envs/superpoint/lib/python3.12/site-packages/keras/src/layers/layer.py:391: UserWarning: `build()` was called on layer 'shared_encoder', however the layer does not have a `build()` method implemented and it looks like it has unbuilt state. This will cause the layer to be marked as built, despite not being actually built, which may cause failures down the line. Make sure to implement a proper `build()` method.
  warnings.warn(
/home/sohaibwaheed/anaconda3/envs/superpoint/lib/python3.12/site-packages/keras/src/layers/layer.py:391: UserWarning: `build()` was called on layer 'interest_point_decoder', however the layer does not have a `build()` method implemented and it looks like it has unbuilt state. This will cause the layer to be marked as built, despite not being actually built, which may cause failures down the line. Make sure to implement a proper `build()` method.
  warnings.warn(


In [ ]:
# Model evaluation

'''datasetIterator = validDataset.__iter__()

test = True
while test:
    tf.config.run_functions_eagerly(False)
    images, points, bins = datasetIterator.__next__()
    modelOutput = magicPoint(images)["finalOutput"]
    tf.config.run_functions_eagerly(True)
    metric.update_state(points, modelOutput)
    for m, value in metric.result().items():
        print(f"{m}: {value.numpy()}")
    metric.reset_state()
    answer = input("Want to evaluate another batch ? (y/n): ")
    if answer.lower() == "y":
        test = True
    else:
        test = False''';

In [7]:
def compute_gradients(images, bins):
    '''
    To get model's current state output along with gradients
    
    Args:
        images [tf.Tensor]: batch of input images
        bins [tf.Tensor]: batch of respective bins present in each image in the batch
    
    Returns:
        output: MagicPoint model output
        gradients [tf.Tensor]: gradients of the current state of the moodel
    '''
    
    # Sample Weights Calculation
    n_points = tf.reduce_sum(tf.where(bins != 64, 1, 0), axis=[1, 2])
    n_points = tf.where(n_points == 0, 300, n_points)

    height, width = MP_INPUT_SHAPE[0] // 8, MP_INPUT_SHAPE[1] // 8
    n_npoints = tf.subtract(height * width, n_points)

    pointsWeights = tf.divide(n_npoints, n_points)

    n_npointsWeights = (1 / (pointsWeights + 1))[..., tf.newaxis, tf.newaxis]
    n_npointsWeights = tf.broadcast_to(n_npointsWeights, shape=[MP_BATCH_SIZE, height, width])

    pointsWeights = (pointsWeights / (pointsWeights + 1))[..., tf.newaxis, tf.newaxis]
    pointsWeights = tf.broadcast_to(pointsWeights, shape=[MP_BATCH_SIZE, height, width])

    sample_weights = tf.where(bins != 64, pointsWeights, n_npointsWeights)

    # Because want to visualize gradients along the training, so performing inferencing with the GradientTape context
    with tf.GradientTape() as tape:
        output = magicPoint(images, training=False)
        loss = magicPoint.compute_loss(
            y=bins,
            y_pred=output["interestPointDecoderOutput"],
            sample_weight=sample_weights,
        )
    
    # Trainable Parameters of First, Middle and Last Convolutional Layers of SE And First And Last Convolutional Layers of IPD
    trainable_vars = magicPoint.layers[2].SEConvBlock_1.conv2d_1.trainable_variables + \
        magicPoint.layers[2].SEConvBlock_2.conv2d_2.trainable_variables + \
        magicPoint.layers[2].SEConvBlock_4.conv2d_2.trainable_variables + \
        magicPoint.layers[3].conv2d.trainable_variables + \
        magicPoint.layers[3].bottleNeckLayer.trainable_variables
        
    gradients = tape.gradient(loss, trainable_vars)
    
    return output, gradients


def plot_to_image(figure):
    '''
    To convert matplotlib figure to tensorflow image so that it can be visualized in tensorboard
    
    Args:
        figure [plt.figure]: matplotlib figure containing all images in a batch
    
    Returns:
        image [tf.Tensor]: figure converted to tensorflow image
    '''
    buffer = io.BytesIO()
    plt.savefig(buffer, format="png")
    plt.close(figure)
    buffer.seek(0)
    image = tf.image.decode_png(buffer.getvalue(), channels=4)
    image = tf.expand_dims(image, 0)
    return image

In [8]:
embeddings_dict = {}
for round in range(roundStart, roundEnd+1):
    # Defining File paths for this iteration
    metadata_fp = os.path.join(train_log_dir, f"metadata_{round}.tsv")
    sprite_fp = os.path.join(train_log_dir, f"sprite_{round}.png")
    
    trainDataset = dataPreparation.loadDataset([dataset_dir + f"/train/train_record_no_{round}.tfrecord"])
    
    print(f"Round {round}/{roundEnd}")
    
    images, points, bins = validDatasetIterator.__next__()
    # To get the shape [batch_size, [bins in each image]]
    flattened_bins = tf.reshape(bins, [MP_BATCH_SIZE, -1])
    imgs_bins = tf.ragged.boolean_mask(flattened_bins, mask=flattened_bins!=64)
    # Saving labels (bins) for each image
    with open(metadata_fp, "w") as f:
        for img_bins in imgs_bins:
            img_bins = list(map(str, img_bins.numpy()))
            if len(img_bins) == 0:
                f.write("None")
            else:
                f.write(",".join(img_bins))
            f.write("\n")
    
    # Generate random images
    # Create a sprite image
    sprite_height, sprite_width = 8, 4 #Square grid dimensions
    sprite_image = tf.Variable(tf.ones((sprite_height * MP_INPUT_SHAPE[0], sprite_width * MP_INPUT_SHAPE[1])))
    # assigning each image to it's respective position in the grid 
    for idx, img in enumerate(images):
        row = idx // sprite_width
        col = idx % sprite_width
        sprite_image[
            row * MP_INPUT_SHAPE[0] : (row + 1) * MP_INPUT_SHAPE[0],
            col * MP_INPUT_SHAPE[1] : (col + 1) * MP_INPUT_SHAPE[1]
        ].assign(img[:, :, 0]/255)

    # Save sprite image
    plt.imsave(sprite_fp, sprite_image, cmap="gray")

    history = magicPoint.fit(
        trainDataset, 
        epochs=1, 
        steps_per_epoch=450, 
        validation_data=validDataset,
        validation_steps=450,
        callbacks=[
            keras.callbacks.ModelCheckpoint(
                filepath=model_dir + f"/magicPoint_{round}.keras",
                monitor="val_loss",
                save_best_only=False,
                save_weights_only=False
            )
        ]
    )
    del trainDataset    
    
    modelOutput, gradients = compute_gradients(images, bins)
    # extracting penultimate layer embeddings for visualization
    embeddings_name = f"embeddings_round_{round}"
    embeddings = tf.Variable(tf.reshape(modelOutput["sharedEncoder"], shape=[MP_BATCH_SIZE, -1]), name=embeddings_name)
    embeddings_dict[embeddings_name] = embeddings
    
    # Add embedding configuration
    embedding = projector_config.embeddings.add()
    embedding.tensor_name = f"embeddings_round_{round}" + "/.ATTRIBUTES/VARIABLE_VALUE"
    embedding.metadata_path = metadata_fp
    
    # Add sprite image configuration (if applicable)
    embedding.sprite.image_path = sprite_fp
    embedding.sprite.single_image_dim.extend(MP_INPUT_SHAPE[-2::-1])
    
    
    # To update CornerDetectionAveragePrecision state
    tf.config.run_functions_eagerly(True)
    metric.update_state(points, modelOutput["finalOutput"])
    tf.config.run_functions_eagerly(False)
    metric_results = metric.result()
    
    
    # Retrieving activations of first convolutional layer of SE and generating it's figure
    random_index = random.randint(0, MP_BATCH_SIZE-1)
    firstLayerActivations = magicPoint.layers[2].SEConvBlock_1.conv2d_1(magicPoint.layers[1](images[random_index:random_index+1]))
    activationsFigure = plt.figure()
    for idx, filter in enumerate(tf.transpose(firstLayerActivations[0], perm=[2, 0, 1]), 1):
        activationsFigure.add_subplot(8, 8, idx)
        plt.imshow(filter, cmap="gray")
        plt.axis("off")
    activationsImage = plot_to_image(activationsFigure)
        
    # Retrieving Kernel of first convolutional layer of SE and generating it's figure
    firstLayerKernel = magicPoint.layers[2].SEConvBlock_1.conv2d_1.kernel
    kernelFigure = plt.figure()
    for idx, filter in enumerate(tf.transpose(firstLayerKernel, perm=[3, 0, 1, 2]), 1):
        kernelFigure.add_subplot(8, 8, idx)
        plt.imshow(filter, cmap="gray")
        plt.axis("off")
    kernelImage = plot_to_image(kernelFigure)
    
    
    with train_summary_writer.as_default():
        tf.summary.scalar('loss', history.history["loss"][0], step=round)
        
        # To write the kernel, bias and gradients of first, middle and layers convolutional layers of Shared Encoder
        tf.summary.histogram("SE First Convolutional Layer Kernel", magicPoint.layers[2].SEConvBlock_1.conv2d_1.kernel, step=round)
        tf.summary.histogram("SE First Convolutional Layer Bias", magicPoint.layers[2].SEConvBlock_1.conv2d_1.bias, step=round)
        tf.summary.histogram("SE First Convolutional Layer Kernel Gradients", gradients[0], step=round)
        tf.summary.histogram("SE First Convolutional Layer Bias Gradients", gradients[1], step=round)
        
        tf.summary.histogram("SE Middle Convolutional Layer Kernel", magicPoint.layers[2].SEConvBlock_2.conv2d_2.kernel, step=round)
        tf.summary.histogram("SE Middle Convolutional Layer Bias", magicPoint.layers[2].SEConvBlock_2.conv2d_2.bias, step=round)
        tf.summary.histogram("SE Middle Convolutional Layer Kernel Gradients", gradients[2], step=round)
        tf.summary.histogram("SE Middle Convolutional Layer Bias Gradients", gradients[3], step=round)
        
        tf.summary.histogram("SE Last Convolutional Layer Kernel", magicPoint.layers[2].SEConvBlock_4.conv2d_2.kernel, step=round)
        tf.summary.histogram("SE Last Convolutional Layer Bias", magicPoint.layers[2].SEConvBlock_4.conv2d_2.bias, step=round)
        tf.summary.histogram("SE Last Convolutional Layer Kernel Gradients", gradients[4], step=round)
        tf.summary.histogram("SE Last Convolutional Layer Bias Gradients", gradients[5], step=round)
        
        
        # To write the kernel and bias of first and last convolutional layer of Interest Point Decoder
        tf.summary.histogram("IPD First Convolutional Layer Kernel", magicPoint.layers[3].conv2d.kernel, step=round)
        tf.summary.histogram("IPD First Convolutional Layer Bias", magicPoint.layers[3].conv2d.bias, step=round)
        tf.summary.histogram("IPD First Convolutional Layer Kernel Gradients", gradients[6], step=round)
        tf.summary.histogram("IPD First Convolutional Layer Bias Gradients", gradients[7], step=round)
        
        tf.summary.histogram("IPD Last Convolutional Layer Kernel", magicPoint.layers[3].bottleNeckLayer.kernel, step=round)
        tf.summary.histogram("IPD Last Convolutional Layer Bias", magicPoint.layers[3].bottleNeckLayer.bias, step=round)
        tf.summary.histogram("IPD Last Convolutional Layer Kernel Gradients", gradients[8], step=round)
        tf.summary.histogram("IPD Last Convolutional Layer Bias Gradients", gradients[9], step=round)
        
        # To write the images of activations of first layer of SE
        tf.summary.image("SE First Layer Activations", activationsImage, step=round)
        # To write the images of Kernel of first layer of SE
        tf.summary.image("SE First Layer Kernel", kernelImage, step=round)        
        
    
        
    with valid_summary_writer.as_default():
        tf.summary.scalar("val_loss", history.history["val_loss"][0], step=round)
        tf.summary.scalar("Mean Average Precision", metric_results["mAP"].numpy(), step=round)
        tf.summary.scalar("Mean Localization Error", metric_results["mLE"].numpy(), step=round)
        
    metric.reset_state()
    
    # Save Embeddings Checkpoint and projector configuration file
    for i in range(1, roundStart):
        embeddings_placeholder = tf.Variable(tf.zeros_like(embeddings), name=f"embeddings_round_{i}")
        embeddings_dict[f"embeddings_round_{i}"] = embeddings_placeholder
    checkpoint = tf.train.Checkpoint(**embeddings_dict)
    # Loading latest checkpoint to restore old embeddings
    latest_checkpoint = tf.train.latest_checkpoint(train_log_dir)
    if not latest_checkpoint is None:
        checkpoint.restore(latest_checkpoint).expect_partial()
        old_ckpt_files = glob(os.path.join(train_log_dir, f"embeddings.ckpt-{round-1}*"))
        os.remove(old_ckpt_files[0])
        os.remove(old_ckpt_files[1])
    checkpoint.save(os.path.join(train_log_dir, f"embeddings.ckpt"))

    # Save configuration file
    projector.visualize_embeddings(train_log_dir, projector_config)

    K.clear_session()
    gc.collect()

Round 409/445


2025-03-12 06:31:43.785619: I tensorflow/core/kernels/data/tf_record_dataset_op.cc:370] TFRecordDataset `buffer_size` is unspecified, default to 262144
I0000 00:00:1741743105.924259    2574 service.cc:148] XLA service 0x7fdcbc06fb40 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1741743105.924562    2574 service.cc:156]   StreamExecutor device (0): NVIDIA GeForce RTX 3070 Laptop GPU, Compute Capability 8.6
2025-03-12 06:31:46.042119: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1741743106.565934    2574 cuda_dnn.cc:529] Loaded cuDNN version 90300


  1/450 ━━━━━━━━━━━━━━━━━━━━ 1:29:47 12s/step - loss: 0.0103

I0000 00:00:1741743116.169291    2574 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


450/450 ━━━━━━━━━━━━━━━━━━━━ 164s 338ms/step - loss: 0.0072 - val_loss: 0.0069
Round 410/445
450/450 ━━━━━━━━━━━━━━━━━━━━ 39s 84ms/step - loss: 0.0072 - val_loss: 0.0068
Round 411/445
450/450 ━━━━━━━━━━━━━━━━━━━━ 30s 65ms/step - loss: 0.0068 - val_loss: 0.0069
Round 412/445
450/450 ━━━━━━━━━━━━━━━━━━━━ 30s 66ms/step - loss: 0.0069 - val_loss: 0.0068
Round 413/445
450/450 ━━━━━━━━━━━━━━━━━━━━ 35s 75ms/step - loss: 0.0070 - val_loss: 0.0069
Round 414/445
450/450 ━━━━━━━━━━━━━━━━━━━━ 31s 67ms/step - loss: 0.0070 - val_loss: 0.0068
Round 415/445
450/450 ━━━━━━━━━━━━━━━━━━━━ 38s 83ms/step - loss: 0.0070 - val_loss: 0.0068
Round 416/445
450/450 ━━━━━━━━━━━━━━━━━━━━ 39s 85ms/step - loss: 0.0071 - val_loss: 0.0068
Round 417/445
450/450 ━━━━━━━━━━━━━━━━━━━━ 31s 67ms/step - loss: 0.0072 - val_loss: 0.0068
Round 418/445
450/450 ━━━━━━━━━━━━━━━━━━━━ 31s 67ms/step - loss: 0.0070 - val_loss: 0.0069
Round 419/445
450/450 ━━━━━━━━━━━━━━━━━━━━ 34s 75ms/step - loss: 0.0071 - val_loss: 0.0068
Round 420/4

2025-03-12 07:12:56.068142: I tensorflow/core/framework/local_rendezvous.cc:405] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]
2025-03-12 07:12:56.068403: I tensorflow/core/framework/local_rendezvous.cc:405] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]
	 [[IteratorGetNext/_6]]
/home/sohaibwaheed/anaconda3/envs/superpoint/lib/python3.12/contextlib.py:158: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(value)


450/450 ━━━━━━━━━━━━━━━━━━━━ 16s 33ms/step - loss: 0.0068 - val_loss: 0.0068


In [9]:
import os
os.system(f"tensorboard --logdir {log_dir}")

2025-03-14 23:19:26.047188: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1741976366.061489    6978 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1741976366.066327    6978 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-03-14 23:19:26.081733: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.

NOTE: Using experimental fast data loading logic. To disable, pass
    "--load_fast=false" and report issues on GitHub. More

2